<a href="https://colab.research.google.com/github/ak55634/Assignment-Visualization/blob/main/My_Meal_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary libraries
!pip install gradio pandas numpy scikit-learn

import gradio as gr
import pandas as pd
import numpy as np
import base64
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Generate synthetic dataset
np.random.seed(42)
data = pd.DataFrame({
    'Age': np.random.randint(18, 65, 500),
    'Weight': np.random.randint(50, 120, 500),
    'Height': np.random.randint(140, 200, 500),
    'Gender': np.random.choice(['Male', 'Female', 'Other'], 500),
    'Race': np.random.choice(['Asian', 'Black', 'White', 'Hispanic', 'Native American', 'Other'], 500),
    'Goals': np.random.choice(['Weight Loss', 'Muscle Gain', 'Maintain Current Weight', 'Improve Overall Health'], 500),
    'Activity_Level': np.random.choice(['Sedentary', 'Lightly Active', 'Moderately Active', 'Very Active'], 500),
    'Dietary_Preference': np.random.choice(['Vegetarian', 'Vegan', 'Keto', 'Balanced', 'Gluten-Free'], 500),
    'Medical_Condition': np.random.choice(['None', 'Diabetes', 'Hypertension', 'High Cholesterol'], 500),
    'Family_History': np.random.choice(['None', 'Heart Disease', 'Obesity', 'Diabetes'], 500),
    'Food_Allergies': np.random.choice(['None', 'Peanuts', 'Dairy', 'Gluten', 'Seafood'], 500),
    'Water_Intake': np.random.randint(1, 5, 500),
    'Alcohol_Consumption': np.random.choice(['Never', 'Occasionally', 'Frequently'], 500),
    'Smoking_Habits': np.random.choice(['Never', 'Occasionally', 'Daily'], 500),
    'Diet_History': np.random.choice(['None', 'Vegetarian', 'Vegan', 'Keto', 'Balanced', 'Gluten-Free'], 500),
    'Current_Medication': np.random.choice(['None', 'Medication A', 'Medication B', 'Medication C'], 500),
    'Labs': np.random.choice(['Normal', 'Elevated', 'Low'], 500),
    'Recommended_Meal': np.random.choice(['Meal A', 'Meal B', 'Meal C', 'Meal D'], 500)
})

# Data preprocessing
data_encoded = pd.get_dummies(data.drop(columns=['Recommended_Meal']), drop_first=True)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Recommended_Meal'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_encoded)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = RandomForestClassifier().fit(X_train, y_train)

# Calculate BMI
def calculate_bmi(weight, height):
    if height > 0:
        return round(weight / ((height / 100) ** 2), 2)
    return "Enter valid height!"

# Save the meal plan in a text file with static sample diet plan notes
def save_meal_plan():
    file_name = "Meal_Plan.txt"
    with open(file_name, "w") as file:
        file.write(" Diet Plan Notes\n")
        file.write("Breakfast: Whole-grain toast with avocado and a side of fresh orange juice\n\n")
        file.write("Mid-Morning Snack: A handful of mixed nuts (almonds, walnuts, and cashews Fresh fruits (like an apple or banana)\n\n")
        file.write("Lunch: Grilled tofu stir-fry with colorful veggies over quinoa\n\n")
        file.write("Dinner: Spaghetti with zucchini noodles, marinara sauce, and parmesan (or plant-based cheese)")
    return file_name

# Function to generate a download link from a given file by reading its content
def get_download_link(file_name, title="Download Meal Plan"):
    with open(file_name, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode("utf-8")
    return f'<a href="data:text/plain;base64,{b64}" download="{file_name}" style="color:#0000ff; font-size:18px; text-decoration:underline;">{title}</a>'

# personalized meal recommendation and generate
def get_personalized_meal(age, weight, height, gender, race, activity_level, dietary_preference,
                          medical_condition, family_history, food_allergies, water_intake,
                          alcohol_consumption, smoking_habits, diet_history, goals, current_medication, labs):
    bmi = calculate_bmi(weight, height)

    # Model prediction
    input_data = pd.DataFrame([[age, weight, height, gender, race, goals, activity_level, dietary_preference,
                                 medical_condition, family_history, food_allergies, water_intake,
                                 alcohol_consumption, smoking_habits, diet_history, current_medication, labs]],
                              columns=['Age', 'Weight', 'Height', 'Gender', 'Race', 'Goals', 'Activity_Level',
                                       'Dietary_Preference', 'Medical_Condition', 'Family_History', 'Food_Allergies',
                                       'Water_Intake', 'Alcohol_Consumption', 'Smoking_Habits', 'Diet_History',
                                       'Current_Medication', 'Labs'])

    input_encoded = pd.get_dummies(input_data, drop_first=True)
    for col in data_encoded.columns:
        if col not in input_encoded.columns:
            input_encoded[col] = 0

    input_scaled = scaler.transform(input_encoded)
    probabilities = model.predict_proba(input_scaled)[0]
    recommended_meal = np.random.choice(label_encoder.classes_, p=probabilities)

    # Save the meal plan (static sample diet plan notes) to a text file
    file_name = save_meal_plan()

    # Generate a download link for the file
    download_link = get_download_link(file_name)

    # download link
    return f"""
    <div style='background-color:#ffe5b4; border-radius:12px; padding:20px; text-align:center;'>
        <p style='color:#0000ff;'>🍽️ Personalized Meal Recommendation 🍽️</p>
        <p style='font-size:22px; font-weight:bold; color:#0000ff;'>{recommended_meal} : {download_link} </p>

    </div>
    """

# Error handling
def validate_inputs(age, weight, height, gender, race, activity_level, dietary_preference,
                    medical_condition, family_history, food_allergies, water_intake,
                    alcohol_consumption, smoking_habits, diet_history, goals, current_medication, labs):
    if weight <= 0 or height <= 0 or water_intake <= 0:
        return """
        <div style='background-color: #ff4c4c; color: white; font-size: 20px; padding: 20px;
                    border-radius: 10px; text-align:center;'>
            ❌ Error: Please enter valid values for <b>Weight, Height, and Water Intake!</b>
        </div>
        """
    return get_personalized_meal(age, weight, height, gender, race, activity_level, dietary_preference,
                                 medical_condition, family_history, food_allergies, water_intake,
                                 alcohol_consumption, smoking_habits, diet_history, goals, current_medication, labs)

# Custom CSS for styling the UI
custom_css = """
body {
    background: #ffe5b4;
    font-family: Arial, sans-serif;
}
h1, h2, label, p {
    color: #0000ff !important;
}
.gradio-button {
    border-radius: 12px;
    background-color: #0000ff;
    color: white;
    font-size: 18px;
    transition: transform 0.2s;
}
.gradio-button:hover {
    transform: scale(1.08);
}
"""

# Create the Gradio interface
with gr.Blocks(css=custom_css) as demo:
    # Header section using the locally uploaded image via data URI:
    gr.Markdown(f"""
    <div style='background-color: #F5F5DC;
                background-position: center;
                background-repeat: no-repeat;
                padding: 30px;
                border-radius: 10px;'>
        <h1 style='text-align:center; color: brown;'>🌟 My Meal Map 🌟</h1>
        <p style='text-align:center; color: brown; font-size: 18px;'>
            your personalized meal recommendations!
        </p>
    </div>
    """)

    # Basic Info and Dynamic BMI Display
    with gr.Row():
        age = gr.Slider(18, 65, value=30, label="🧑 Age", interactive=True)
        weight = gr.Number(label="⚖️ Weight (kg)", interactive=True)
        height = gr.Number(label="📏 Height (cm)", interactive=True)
        bmi_display = gr.Textbox(label="🏆 BMI", interactive=False)

    weight.change(calculate_bmi, inputs=[weight, height], outputs=bmi_display)
    height.change(calculate_bmi, inputs=[weight, height], outputs=bmi_display)

    # Detailed User Inputs
    with gr.Row():
        gender = gr.Dropdown(["Male", "Female", "Other"], label="🚻 Gender")
        race = gr.Dropdown(["Asian", "Black", "White", "Hispanic", "Native American", "Other"], label="🌍 Race")
        goals = gr.Dropdown(["Weight Loss", "Muscle Gain", "Maintain Current Weight", "Improve Overall Health"], label="🎯 Goals")
        activity_level = gr.Dropdown(["Sedentary", "Lightly Active", "Moderately Active", "Very Active"], label="🏃 Activity Level")
        water_intake = gr.Number(label="💧 Water Intake (Liters/Day)", interactive=True)
        dietary_preference = gr.Dropdown(["Vegetarian", "Vegan", "Keto", "Balanced", "Gluten-Free"], label="🥗 Dietary Preference")
        medical_condition = gr.Dropdown(["None", "Diabetes", "Hypertension", "High Cholesterol"], label="💊 Medical Condition")
        family_history = gr.Dropdown(["None", "Heart Disease", "Obesity", "Diabetes"], label="🏥 Family History")
        food_allergies = gr.Dropdown(["None", "Peanuts", "Dairy", "Gluten", "Seafood"], label="🚫 Food Allergies")
        alcohol_consumption = gr.Dropdown(["Never", "Occasionally", "Frequently"], label="🍷 Alcohol Consumption")
        smoking_habits = gr.Dropdown(["Never", "Occasionally", "Daily"], label="🚬 Smoking Habits")
        diet_history = gr.Dropdown(["None", "Vegetarian", "Vegan", "Keto", "Balanced", "Gluten-Free"], label="🍽️ Diets Tried Before")
        current_medication = gr.Textbox(label="💊 Current Medication", interactive=True)
        labs = gr.Textbox(label="🧪 Labs", interactive=True)

    # Button to trigger recommendation and output
    submit_button = gr.Button("Get Recommendation")
    result_output = gr.HTML()

    submit_button.click(
        validate_inputs,
        inputs=[age, weight, height, gender, race, activity_level, dietary_preference,
                medical_condition, family_history, food_allergies, water_intake,
                alcohol_consumption, smoking_habits, diet_history, goals, current_medication, labs],
        outputs=result_output
    )

# Launch the Gradio app with a shareable link
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://483fd0280a5cb8a9a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
